In [1]:
from FlagEmbedding import BGEM3FlagModel
import os

os.environ['CURL_CA_BUNDLE'] = ''

In [2]:
model = BGEM3FlagModel("BAAI/bge-m3",
                       use_fp16=True)

/home/murad/anaconda3/envs/rag/lib/python3.10/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/murad/anaconda3/envs/rag/lib/python3.10/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [3]:
sentences_1 = ["Python nədir?", "Böyük dil modelləri nədir?"]
sentences_2 = ["Mənim adım Muraddır və Maşın Öyrənməsi mühəndisiyəm.", 
               "Böyük dil modeli insan dilini yarada bilən və əlaqəli vəzifələri yerinə yetirə bilən bir süni intellekt növüdür.."]

In [10]:
embeddings_1 = model.encode(sentences_1, 
                            batch_size=12, 
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

[[0.37017542 0.39612377]
 [0.30685663 0.6944343 ]]


In [8]:
tokens = model.tokenizer(text=sentences_1)

In [14]:
from transformers import AutoTokenizer

# Load the BGE-M3 tokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# Text to tokenize
text = "Mənim adım Muraddır və Maşın Öyrənməsi mühəndisiyəm."

# Tokenize the text
tokens = tokenizer.tokenize(text)

# Print the tokens
print(tokens)

# If you want to see the token IDs
token_ids = tokenizer.encode(text)
print(token_ids)

# To see the exact boundaries in the original text, you can use
encoded = tokenizer(text, return_offsets_mapping=True)
for token, (start, end) in zip(tokens, encoded["offset_mapping"]):
    print(f"Token: {token}, Original text: '{text[start:end]}'")

/home/murad/anaconda3/envs/rag/lib/python3.10/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['▁Mənim', '▁adım', '▁Murad', 'dır', '▁və', '▁Ma', 'şın', '▁Ö', 'yr', 'ən', 'məsi', '▁mühəndis', 'iy', 'əm', '.']
[0, 177565, 54854, 148674, 2544, 530, 911, 83199, 5973, 12271, 7879, 28932, 234142, 12712, 10857, 5, 2]
Token: ▁Mənim, Original text: ''
Token: ▁adım, Original text: 'Mənim'
Token: ▁Murad, Original text: ' adım'
Token: dır, Original text: ' Murad'
Token: ▁və, Original text: 'dır'
Token: ▁Ma, Original text: ' və'
Token: şın, Original text: ' Ma'
Token: ▁Ö, Original text: 'şın'
Token: yr, Original text: ' Ö'
Token: ən, Original text: 'yr'
Token: məsi, Original text: 'ən'
Token: ▁mühəndis, Original text: 'məsi'
Token: iy, Original text: ' mühəndis'
Token: əm, Original text: 'iy'
Token: ., Original text: 'əm'


In [18]:
# Access the tokenizer from the model
tokenizer = model.tokenizer

# Text to tokenize
text = "ASAN xidmət mərkəzlərinə xoş gəlmisiniz!"

# Tokenize the text
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# Get token IDs
token_ids = tokenizer.encode(text, add_special_tokens=False)
print("Token IDs:", token_ids)

# If you want to see the mapping between tokens and original text
# We need to get the offsets manually as FlagEmbedding might not have direct support
# for offset_mapping like transformers
encoded_plus = tokenizer.encode_plus(text, return_offsets_mapping=True)
if "offset_mapping" in encoded_plus:
    # If offset_mapping is supported
    for token, (start, end) in zip(tokens, encoded_plus["offset_mapping"]):
        if start != end:  # Skip special tokens which might have (0,0) offsets
            print(f"Token: {token}, Original text: '{text[start:end]}'")
else:
    # Alternative approach if offset_mapping isn't directly supported
    print("Note: Direct offset mapping not available in this implementation.")
    # Display tokens with their positions in the sequence
    for i, token in enumerate(tokens):
        print(f"Position {i}: Token: {token}")

Tokens: ['▁', 'ASAN', '▁xidmət', '▁mərkəzləri', 'nə', '▁xoş', '▁gəl', 'mi', 'siniz', '!']
Token IDs: [6, 95239, 25058, 212523, 5022, 129395, 66366, 266, 25098, 38]
Token: ASAN, Original text: 'A'
Token: ▁xidmət, Original text: 'ASAN'
Token: ▁mərkəzləri, Original text: ' xidmət'
Token: nə, Original text: ' mərkəzləri'
Token: ▁xoş, Original text: 'nə'
Token: ▁gəl, Original text: ' xoş'
Token: mi, Original text: ' gəl'
Token: siniz, Original text: 'mi'
Token: !, Original text: 'siniz'
